<a href="https://colab.research.google.com/github/MLDreamer/AIMathematicallyexplained/blob/main/Geometry_of_Thought_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# The Geometry of Thought: Interactive Playground
# Companion code for "The Geometry of Thought" Medium article
# Author: DrSwarnenduAI

# ============================================================================
# SETUP AND INSTALLATIONS
# ============================================================================

!pip install numpy matplotlib plotly scikit-learn umap-learn wordcloud seaborn
!pip install sentence-transformers transformers torch
!pip install gensim
!pip install -q ipywidgets

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# Chapter 1: The Magic of Word Vectors
# ============================================================================

print("🎯 THE GEOMETRY OF THOUGHT: INTERACTIVE PLAYGROUND")
print("=" * 60)
print("Exploring how meaning becomes mathematics in high-dimensional space")
print()

class GeometryOfThought:
    def __init__(self):
        """Initialize the geometric thought explorer"""
        print("🔄 Loading semantic model...")
        # Using a smaller, faster model for Colab
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.embedding_dim = 384  # Dimension of our semantic space
        print(f"✅ Model loaded! Working in {self.embedding_dim}-dimensional space")

    def get_word_vector(self, word):
        """Convert a word to its vector representation"""
        return self.model.encode([word])[0]

    def demonstrate_basic_geometry(self):
        """Show how words become points in space"""
        print("\n🔍 CHAPTER 1: Words as Points in Space")
        print("-" * 40)

        # Sample words
        words = ['king', 'queen', 'man', 'woman', 'prince', 'princess',
                'boy', 'girl', 'father', 'mother']

        # Get embeddings
        embeddings = self.model.encode(words)

        # Reduce to 2D for visualization
        pca = PCA(n_components=2)
        embeddings_2d = pca.fit_transform(embeddings)

        # Create visualization
        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=embeddings_2d[:, 0],
            y=embeddings_2d[:, 1],
            mode='markers+text',
            text=words,
            textposition="middle right",
            marker=dict(size=12, color='blue'),
            name='Words'
        ))

        fig.update_layout(
            title="Words as Points in 2D Semantic Space<br><sub>Each word has precise coordinates based on meaning</sub>",
            xaxis_title="Semantic Dimension 1",
            yaxis_title="Semantic Dimension 2",
            width=800, height=600
        )

        fig.show()

        # Show the actual high-dimensional coordinates
        print(f"\nExample: The word 'king' in {self.embedding_dim}D space:")
        king_vector = self.get_word_vector('king')
        print(f"First 10 dimensions: {king_vector[:10]}")
        print(f"Vector magnitude: {np.linalg.norm(king_vector):.3f}")

        return embeddings, words

    def demonstrate_vector_arithmetic(self):
        """Show the famous King - Man + Woman = Queen calculation"""
        print("\n🧮 CHAPTER 2: The Arithmetic of Meaning")
        print("-" * 40)

        # Get vectors for our famous equation
        king = self.get_word_vector('king')
        man = self.get_word_vector('man')
        woman = self.get_word_vector('woman')
        queen = self.get_word_vector('queen')

        # Perform the calculation
        result = king - man + woman

        # Calculate similarities
        similarity_to_queen = cosine_similarity([result], [queen])[0][0]

        print(f"Vector arithmetic: King - Man + Woman")
        print(f"Result similarity to 'Queen': {similarity_to_queen:.4f}")
        print(f"(1.0 = identical, 0.0 = unrelated)")

        # Test with more examples
        test_words = ['queen', 'princess', 'woman', 'lady', 'duchess']
        similarities = []

        for word in test_words:
            word_vector = self.get_word_vector(word)
            sim = cosine_similarity([result], [word_vector])[0][0]
            similarities.append(sim)

        # Create bar chart
        fig = go.Figure(data=[
            go.Bar(x=test_words, y=similarities,
                   marker_color=['red' if word == 'queen' else 'blue' for word in test_words])
        ])

        fig.update_layout(
            title="King - Man + Woman: Which word is closest?<br><sub>Red bar shows the mathematical 'answer'</sub>",
            xaxis_title="Test Words",
            yaxis_title="Cosine Similarity",
            width=800, height=400
        )

        fig.show()

        return result, similarities

    def explore_semantic_relationships(self):
        """Explore different types of semantic relationships"""
        print("\n🔗 CHAPTER 3: Discovering Relationship Patterns")
        print("-" * 40)

        # Define relationship sets
        relationships = {
            'Gender': [('king', 'queen'), ('man', 'woman'), ('boy', 'girl'),
                      ('prince', 'princess'), ('actor', 'actress')],
            'Comparative': [('good', 'better'), ('big', 'bigger'), ('fast', 'faster'),
                           ('smart', 'smarter'), ('tall', 'taller')],
            'Capital-Country': [('paris', 'france'), ('london', 'england'),
                               ('tokyo', 'japan'), ('berlin', 'germany')],
            'Plural': [('cat', 'cats'), ('dog', 'dogs'), ('book', 'books'),
                      ('car', 'cars'), ('house', 'houses')]
        }

        relationship_vectors = {}

        for rel_type, pairs in relationships.items():
            vectors = []
            for word1, word2 in pairs:
                vec1 = self.get_word_vector(word1)
                vec2 = self.get_word_vector(word2)
                relationship_vector = vec2 - vec1
                vectors.append(relationship_vector)

            # Average the relationship vectors
            avg_relationship = np.mean(vectors, axis=0)
            relationship_vectors[rel_type] = avg_relationship

            # Calculate consistency (how similar are the relationship vectors?)
            similarities = []
            for vec in vectors:
                sim = cosine_similarity([vec], [avg_relationship])[0][0]
                similarities.append(sim)

            avg_consistency = np.mean(similarities)
            print(f"{rel_type} relationship consistency: {avg_consistency:.4f}")

        # Test the relationships with new examples
        print("\n🎯 Testing Learned Relationships:")

        # Test gender relationship
        father = self.get_word_vector('father')
        predicted_mother = father + relationship_vectors['Gender']
        actual_mother = self.get_word_vector('mother')
        accuracy = cosine_similarity([predicted_mother], [actual_mother])[0][0]
        print(f"Father + Gender_Vector → Mother: {accuracy:.4f} similarity")

        # Test comparative relationship
        slow = self.get_word_vector('slow')
        predicted_slower = slow + relationship_vectors['Comparative']
        actual_slower = self.get_word_vector('slower')
        accuracy = cosine_similarity([predicted_slower], [actual_slower])[0][0]
        print(f"Slow + Comparative_Vector → Slower: {accuracy:.4f} similarity")

        return relationship_vectors

    def visualize_semantic_neighborhoods(self):
        """Show how related concepts cluster together"""
        print("\n🏘️ CHAPTER 4: Semantic Neighborhoods")
        print("-" * 40)

        # Define semantic categories
        categories = {
            'Animals': ['dog', 'cat', 'lion', 'elephant', 'tiger', 'bear'],
            'Colors': ['red', 'blue', 'green', 'yellow', 'purple', 'orange'],
            'Emotions': ['happy', 'sad', 'angry', 'excited', 'calm', 'nervous'],
            'Professions': ['doctor', 'teacher', 'engineer', 'lawyer', 'nurse', 'chef'],
            'Transportation': ['car', 'plane', 'train', 'bike', 'boat', 'bus']
        }

        # Get all embeddings
        all_words = []
        category_labels = []
        colors = ['red', 'blue', 'green', 'orange', 'purple']

        for i, (category, words) in enumerate(categories.items()):
            all_words.extend(words)
            category_labels.extend([category] * len(words))

        embeddings = self.model.encode(all_words)

        # Reduce to 2D with UMAP (better for clusters than PCA)
        reducer = umap.UMAP(n_components=2, random_state=42)
        embeddings_2d = reducer.fit_transform(embeddings)

        # Create DataFrame for plotting
        df = pd.DataFrame({
            'x': embeddings_2d[:, 0],
            'y': embeddings_2d[:, 1],
            'word': all_words,
            'category': category_labels
        })

        # Create interactive plot
        fig = px.scatter(df, x='x', y='y', color='category',
                        hover_data=['word'],
                        title="Semantic Neighborhoods: Related Concepts Cluster Together")

        # Add text annotations
        for i, row in df.iterrows():
            fig.add_annotation(
                x=row['x'], y=row['y'],
                text=row['word'],
                showarrow=False,
                font=dict(size=10)
            )

        fig.update_layout(width=900, height=700)
        fig.show()

        return embeddings, all_words, category_labels

    def demonstrate_creative_combinations(self):
        """Show how AI can be 'creative' through geometric interpolation"""
        print("\n🎨 CHAPTER 5: Creativity Through Geometry")
        print("-" * 40)

        # Creative combinations through vector interpolation
        concepts = {
            'Technology': self.get_word_vector('computer'),
            'Nature': self.get_word_vector('tree'),
            'Art': self.get_word_vector('painting'),
            'Music': self.get_word_vector('song'),
            'Food': self.get_word_vector('cake')
        }

        # Find creative combinations
        combinations = []
        for name1, vec1 in concepts.items():
            for name2, vec2 in concepts.items():
                if name1 < name2:  # Avoid duplicates
                    # Interpolate between the two concepts
                    interpolated = (vec1 + vec2) / 2
                    combinations.append((f"{name1} + {name2}", interpolated))

        # Find closest real words to these creative combinations
        test_words = ['website', 'sculpture', 'recipe', 'algorithm', 'garden',
                     'melody', 'design', 'innovation', 'harmony', 'creation']

        print("Creative Concept Combinations:")
        print("-" * 30)

        for combo_name, combo_vector in combinations[:3]:  # Show first 3
            similarities = []
            for word in test_words:
                word_vec = self.get_word_vector(word)
                sim = cosine_similarity([combo_vector], [word_vec])[0][0]
                similarities.append((word, sim))

            # Sort by similarity
            similarities.sort(key=lambda x: x[1], reverse=True)
            top_match = similarities[0]

            print(f"{combo_name} → '{top_match[0]}' (similarity: {top_match[1]:.3f})")

    def business_applications_demo(self):
        """Demonstrate business applications of semantic geometry"""
        print("\n💼 CHAPTER 6: Business Applications")
        print("-" * 40)

        # Simulate a company knowledge base
        documents = [
            "Our customer satisfaction scores have improved significantly this quarter",
            "Client happiness metrics show positive trends across all departments",
            "User experience ratings indicate strong product-market fit",
            "Financial performance exceeded expectations with 15% growth",
            "Revenue streams diversified successfully into new market segments",
            "Product development timeline accelerated through agile methodologies",
            "Innovation pipeline includes AI-powered features for next release"
        ]

        # Query examples
        queries = [
            "How are customers feeling about our products?",
            "What is our financial situation?",
            "What innovations are we working on?"
        ]

        print("Semantic Search Demo:")
        print("Finding relevant documents using meaning, not keywords")
        print()

        # Get document embeddings
        doc_embeddings = self.model.encode(documents)

        for query in queries:
            print(f"Query: '{query}'")
            query_embedding = self.model.encode([query])

            # Calculate similarities
            similarities = cosine_similarity(query_embedding, doc_embeddings)[0]

            # Find top match
            best_match_idx = np.argmax(similarities)
            best_similarity = similarities[best_match_idx]
            best_document = documents[best_match_idx]

            print(f"Best match (similarity: {best_similarity:.3f}):")
            print(f"→ '{best_document}'")
            print()

    def run_full_demonstration(self):
        """Run the complete geometry of thought demonstration"""
        print("🚀 Starting Complete Demonstration...")
        print()

        # Run all demonstrations
        self.demonstrate_basic_geometry()
        self.demonstrate_vector_arithmetic()
        self.explore_semantic_relationships()
        self.visualize_semantic_neighborhoods()
        self.demonstrate_creative_combinations()
        self.business_applications_demo()

        print("🎉 DEMONSTRATION COMPLETE!")
        print("-" * 60)
        print("Key Insights:")
        print("• Words are points in high-dimensional space")
        print("• Relationships become vectors (direction + magnitude)")
        print("• Analogies work through vector arithmetic")
        print("• Similar concepts cluster geometrically")
        print("• Creativity emerges from exploring new regions")
        print("• Business applications leverage semantic similarity")
        print()
        print("The mathematics of meaning is beautiful and precise! 🤖✨")

# ============================================================================
# INTERACTIVE EXPLORATION FUNCTIONS
# ============================================================================

def explore_custom_analogies():
    """Let users test their own analogies"""
    print("\n🧪 CUSTOM ANALOGY EXPLORER")
    print("-" * 30)
    print("Test your own analogies using the format: A is to B as C is to ?")
    print("Example: 'paris france london' finds the word most similar to London+Paris-France")
    print()

    # This would be interactive in actual Colab
    # For demo purposes, showing structure
    example_analogies = [
        ('paris', 'france', 'london', 'england'),
        ('swimming', 'water', 'flying', 'air'),
        ('pen', 'write', 'brush', 'paint')
    ]

    geo_explorer = GeometryOfThought()

    for a, b, c, expected in example_analogies:
        vec_a = geo_explorer.get_word_vector(a)
        vec_b = geo_explorer.get_word_vector(b)
        vec_c = geo_explorer.get_word_vector(c)

        # Calculate A is to B as C is to ?
        result_vector = vec_b - vec_a + vec_c
        expected_vector = geo_explorer.get_word_vector(expected)

        similarity = cosine_similarity([result_vector], [expected_vector])[0][0]
        print(f"{a}:{b} :: {c}:{expected} → Similarity: {similarity:.3f}")

def create_semantic_map():
    """Create a personalized semantic map"""
    print("\n🗺️ PERSONAL SEMANTIC MAP CREATOR")
    print("-" * 35)

    # Example personal interests
    interests = ['programming', 'music', 'travel', 'cooking', 'reading',
                'exercise', 'movies', 'art', 'science', 'games']

    geo_explorer = GeometryOfThought()
    embeddings = geo_explorer.model.encode(interests)

    # Reduce to 2D
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings)

    # Create visualization
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=embeddings_2d[:, 0],
        y=embeddings_2d[:, 1],
        mode='markers+text',
        text=interests,
        textposition="middle right",
        marker=dict(size=15, color='purple'),
        name='Interests'
    ))

    fig.update_layout(
        title="Your Personal Semantic Space<br><sub>How your interests relate geometrically</sub>",
        xaxis_title="Semantic Dimension 1",
        yaxis_title="Semantic Dimension 2",
        width=800, height=600
    )

    fig.show()

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    # Initialize and run the complete demonstration
    geometry_explorer = GeometryOfThought()
    geometry_explorer.run_full_demonstration()

    # Additional interactive features
    explore_custom_analogies()
    create_semantic_map()

    print("\n" + "="*60)
    print("🔬 EXPERIMENT FURTHER:")
    print("• Modify the word lists to explore different semantic relationships")
    print("• Try your own vector arithmetic combinations")
    print("• Test business documents from your own domain")
    print("• Explore how different languages create similar geometric patterns")
    print("="*60)



🎯 THE GEOMETRY OF THOUGHT: INTERACTIVE PLAYGROUND
Exploring how meaning becomes mathematics in high-dimensional space

🔄 Loading semantic model...
✅ Model loaded! Working in 384-dimensional space
🚀 Starting Complete Demonstration...


🔍 CHAPTER 1: Words as Points in Space
----------------------------------------



Example: The word 'king' in 384D space:
First 10 dimensions: [-0.05959929  0.0505124  -0.06951005  0.07968026 -0.04674774  0.00098879
  0.0790433  -0.01273934  0.05839581 -0.03140248]
Vector magnitude: 1.000

🧮 CHAPTER 2: The Arithmetic of Meaning
----------------------------------------
Vector arithmetic: King - Man + Woman
Result similarity to 'Queen': 0.5795
(1.0 = identical, 0.0 = unrelated)



🔗 CHAPTER 3: Discovering Relationship Patterns
----------------------------------------
Gender relationship consistency: 0.6679
Comparative relationship consistency: 0.6085
Capital-Country relationship consistency: 0.8501
Plural relationship consistency: 0.7535

🎯 Testing Learned Relationships:
Father + Gender_Vector → Mother: 0.7555 similarity
Slow + Comparative_Vector → Slower: 0.9012 similarity

🏘️ CHAPTER 4: Semantic Neighborhoods
----------------------------------------



🎨 CHAPTER 5: Creativity Through Geometry
----------------------------------------
Creative Concept Combinations:
------------------------------
Nature + Technology → 'website' (similarity: 0.465)
Art + Technology → 'design' (similarity: 0.516)
Art + Nature → 'sculpture' (similarity: 0.554)

💼 CHAPTER 6: Business Applications
----------------------------------------
Semantic Search Demo:
Finding relevant documents using meaning, not keywords

Query: 'How are customers feeling about our products?'
Best match (similarity: 0.569):
→ 'User experience ratings indicate strong product-market fit'

Query: 'What is our financial situation?'
Best match (similarity: 0.398):
→ 'Financial performance exceeded expectations with 15% growth'

Query: 'What innovations are we working on?'
Best match (similarity: 0.443):
→ 'Innovation pipeline includes AI-powered features for next release'

🎉 DEMONSTRATION COMPLETE!
------------------------------------------------------------
Key Insights:
• Words are po


🔬 EXPERIMENT FURTHER:
• Modify the word lists to explore different semantic relationships
• Try your own vector arithmetic combinations
• Test business documents from your own domain
• Explore how different languages create similar geometric patterns


In [ ]:
# ============================================================================
# ANIMATED SEMANTIC JOURNEY GENERATOR
# Creates GIF visualizations of concepts moving through semantic space
# ============================================================================

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
import seaborn as sns
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer
import io
from PIL import Image
import base64

class SemanticAnimationGenerator:
    """Generate animated visualizations of semantic relationships"""

    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.fig_size = (12, 8)
        self.dpi = 100

    def create_word_journey_animation(self, word_sequence, title="Semantic Journey"):
        """
        Create animated visualization of words moving through semantic space

        Args:
            word_sequence: List of words showing conceptual progression
            title: Animation title
        Returns:
            Animation object and frame data for GIF creation
        """

        # Get embeddings for all words
        embeddings = self.model.encode(word_sequence)

        # Reduce to 2D for visualization
        pca = PCA(n_components=2)
        coords_2d = pca.fit_transform(embeddings)

        # Set up the plot
        fig, ax = plt.subplots(figsize=self.fig_size, dpi=self.dpi)
        ax.set_xlim(coords_2d[:, 0].min() - 0.5, coords_2d[:, 0].max() + 0.5)
        ax.set_ylim(coords_2d[:, 1].min() - 0.5, coords_2d[:, 1].max() + 0.5)

        # Style the plot
        ax.set_facecolor('black')
        fig.patch.set_facecolor('black')
        ax.grid(True, alpha=0.3, color='white')
        ax.set_title(title, color='white', fontsize=16, fontweight='bold')
        ax.set_xlabel('Semantic Dimension 1', color='white', fontsize=12)
        ax.set_ylabel('Semantic Dimension 2', color='white', fontsize=12)
        ax.tick_params(colors='white')

        # Animation data structures
        points = []
        labels = []
        trails = []

        def init():
            """Initialize animation"""
            return points + labels + trails

        def animate(frame):
            """Animation function for each frame"""
            # Clear previous frame
            ax.clear()
            ax.set_xlim(coords_2d[:, 0].min() - 0.5, coords_2d[:, 0].max() + 0.5)
            ax.set_ylim(coords_2d[:, 1].min() - 0.5, coords_2d[:, 1].max() + 0.5)
            ax.set_facecolor('black')
            ax.grid(True, alpha=0.3, color='white')
            ax.set_title(title, color='white', fontsize=16, fontweight='bold')
            ax.set_xlabel('Semantic Dimension 1', color='white', fontsize=12)
            ax.set_ylabel('Semantic Dimension 2', color='white', fontsize=12)
            ax.tick_params(colors='white')

            # Show trail of previous points with fading effect
            for i in range(min(frame + 1, len(coords_2d))):
                alpha = 0.3 + 0.7 * (i / max(1, frame))  # Fade older points
                size = 50 + 100 * (i / max(1, len(coords_2d) - 1))  # Grow with progression

                # Color progression from blue to red
                color_intensity = i / max(1, len(coords_2d) - 1)
                color = plt.cm.plasma(color_intensity)

                ax.scatter(coords_2d[i, 0], coords_2d[i, 1],
                          s=size, alpha=alpha, c=[color],
                          edgecolors='white', linewidth=2)

                # Add word labels
                ax.annotate(word_sequence[i],
                           (coords_2d[i, 0], coords_2d[i, 1]),
                           xytext=(10, 10), textcoords='offset points',
                           fontsize=12, color='white', fontweight='bold',
                           bbox=dict(boxstyle='round,pad=0.3',
                                   facecolor=color, alpha=0.8))

                # Draw connections between consecutive points
                if i > 0:
                    ax.plot([coords_2d[i-1, 0], coords_2d[i, 0]],
                           [coords_2d[i-1, 1], coords_2d[i, 1]],
                           color='white', alpha=0.6, linewidth=2)

            # Add current frame indicator
            frame_text = f"Step {min(frame + 1, len(coords_2d))}/{len(coords_2d)}"
            ax.text(0.02, 0.98, frame_text, transform=ax.transAxes,
                   fontsize=14, color='yellow', fontweight='bold',
                   verticalalignment='top')

            return []

        # Create animation
        anim = animation.FuncAnimation(
            fig, animate, init_func=init,
            frames=len(coords_2d) + 10,  # Extra frames to show final state
            interval=1000, blit=False, repeat=True
        )

        return anim, coords_2d, word_sequence

    def create_analogy_visualization(self, analogy_parts, result_word):
        """
        Visualize vector arithmetic: A - B + C = D

        Args:
            analogy_parts: tuple (A, B, C) for "A is to B as C is to D"
            result_word: The expected result D
        """

        a_word, b_word, c_word = analogy_parts
        words = [a_word, b_word, c_word, result_word]

        # Get embeddings
        embeddings = self.model.encode(words)

        # Calculate result vector
        a_vec, b_vec, c_vec, d_vec = embeddings
        calculated_result = a_vec - b_vec + c_vec

        # Add calculated result to visualization
        all_embeddings = np.vstack([embeddings, calculated_result.reshape(1, -1)])
        all_words = words + [f"Calculated Result"]

        # Reduce to 2D
        pca = PCA(n_components=2)
        coords_2d = pca.fit_transform(all_embeddings)

        # Create animation showing the vector arithmetic
        fig, ax = plt.subplots(figsize=self.fig_size, dpi=self.dpi)

        def animate_analogy(frame):
            ax.clear()
            ax.set_facecolor('black')
            fig.patch.set_facecolor('black')

            # Set title based on frame
            titles = [
                f"Step 1: Start with '{a_word}'",
                f"Step 2: Subtract '{b_word}' relationship",
                f"Step 3: Add '{c_word}' relationship",
                f"Step 4: Result closest to '{result_word}'!"
            ]

            title_idx = min(frame, len(titles) - 1)
            ax.set_title(titles[title_idx], color='white', fontsize=16, fontweight='bold')

            # Show points progressively
            colors = ['red', 'blue', 'green', 'gold', 'purple']

            for i in range(min(frame + 1, len(coords_2d))):
                ax.scatter(coords_2d[i, 0], coords_2d[i, 1],
                          s=200, c=colors[i], alpha=0.8,
                          edgecolors='white', linewidth=2)

                ax.annotate(all_words[i],
                           (coords_2d[i, 0], coords_2d[i, 1]),
                           xytext=(15, 15), textcoords='offset points',
                           fontsize=12, color='white', fontweight='bold',
                           bbox=dict(boxstyle='round,pad=0.5',
                                   facecolor=colors[i], alpha=0.8))

            # Draw vectors for arithmetic operations
            if frame >= 1:  # Show A to B vector
                ax.annotate('', xy=coords_2d[1], xytext=coords_2d[0],
                           arrowprops=dict(arrowstyle='->', color='yellow', lw=3))
                ax.text((coords_2d[0, 0] + coords_2d[1, 0])/2,
                       (coords_2d[0, 1] + coords_2d[1, 1])/2,
                       'subtract', color='yellow', fontweight='bold')

            if frame >= 2:  # Show C vector
                ax.annotate('', xy=coords_2d[2], xytext=coords_2d[4] if frame >= 4 else coords_2d[1],
                           arrowprops=dict(arrowstyle='->', color='cyan', lw=3))
                ax.text(coords_2d[2, 0], coords_2d[2, 1] - 0.3,
                       'add', color='cyan', fontweight='bold')

            if frame >= 3:  # Show final result
                ax.plot([coords_2d[3, 0], coords_2d[4, 0]],
                       [coords_2d[3, 1], coords_2d[4, 1]],
                       'white', linestyle='--', linewidth=2)

                distance = np.linalg.norm(coords_2d[3] - coords_2d[4])
                ax.text((coords_2d[3, 0] + coords_2d[4, 0])/2,
                       (coords_2d[3, 1] + coords_2d[4, 1])/2,
                       f'distance: {distance:.2f}',
                       color='white', fontweight='bold',
                       bbox=dict(boxstyle='round', facecolor='black', alpha=0.8))

            ax.grid(True, alpha=0.3, color='white')
            ax.set_xlabel('Semantic Dimension 1', color='white')
            ax.set_ylabel('Semantic Dimension 2', color='white')
            ax.tick_params(colors='white')

            return []

        anim = animation.FuncAnimation(
            fig, animate_analogy, frames=8, interval=1500,
            repeat=True, blit=False
        )

        return anim

    def create_concept_evolution_gif(self, concept_chain, filename="semantic_evolution.gif"):
        """
        Create and save GIF showing concept evolution

        Args:
            concept_chain: List of related concepts showing progression
            filename: Output filename for GIF
        """

        anim, coords, words = self.create_word_journey_animation(
            concept_chain, "Concept Evolution Through Semantic Space"
        )

        # Save as GIF
        writer = animation.PillowWriter(fps=1)
        anim.save(filename, writer=writer)

        print(f"🎬 Animation saved as {filename}")
        print(f"📊 Visualized {len(words)} concepts across {len(coords)} dimensions")

        return filename

    def create_business_transformation_animation(self):
        """Create animation showing business concept transformation"""

        # Business transformation journey
        transformation_stages = [
            'startup', 'growth', 'scaling', 'optimization',
            'innovation', 'disruption', 'market_leader'
        ]

        return self.create_word_journey_animation(
            transformation_stages,
            "Business Evolution in Semantic Space"
        )

# ============================================================================
# USAGE EXAMPLES
# ============================================================================

def generate_sample_animations():
    """Generate sample animations for different concepts"""

    generator = SemanticAnimationGenerator()

    # Example 1: Technology evolution
    tech_evolution = ['abacus', 'calculator', 'computer', 'smartphone', 'ai']
    anim1 = generator.create_concept_evolution_gif(
        tech_evolution, "tech_evolution.gif"
    )

    # Example 2: Learning progression
    learning_stages = ['curiosity', 'study', 'practice', 'mastery', 'teaching']
    anim2 = generator.create_concept_evolution_gif(
        learning_stages, "learning_journey.gif"
    )

    # Example 3: King-Queen analogy visualization
    analogy_anim = generator.create_analogy_visualization(
        ('king', 'man', 'woman'), 'queen'
    )

    print("🎯 Sample animations generated!")
    print("Use these patterns to create your own semantic journey visualizations")

    return anim1, anim2, analogy_anim

if __name__ == "__main__":
    generate_sample_animations()